## Parser

In [1]:
HEIGHT = 284
WIDTH = 284

In [ ]:
feature_description = {
    'eventId': tf.io.FixedLenFeature([1], tf.int64, default_value=0),
    'image': tf.io.FixedLenFeature([3], tf.string, default_value=["",]*3)
}

feature_shape = {
    'eventId': tf.TensorShape([1,]),
    'image': tf.TensorShape([568, 568, 2])
}

def parser_fn(proto):
    serialized = tf.io.parse_single_example(proto, feature_description)
    deserialized = {k: (tf.sparse.to_dense(sparse_ops.deserialize_sparse(v, K.floatx()))
                        if k != 'eventId' else v) for k,v in serialized.items()}
    [deserialized[k].set_shape(feature_shape[k]) for k in deserialized.keys()]
    x = deserialized['image']
    x = tf.expand_dims(x, axis=0)
    x = K.pool2d(x, pool_size=(2, 2), strides=(2, 2), pool_mode='avg')
    # sum instead of avg
    x = 4.*x
    deserialized['image'] = tf.squeeze(x)
    
    return deserialized

In [ ]:
DATA_DIR = ''
BATCH_SIZE = 16
BUFFER_SIZE = 10

filename = [os.path.join(DATA_DIR, 'B4.tfrecord')]
dataset = tf.data.TFRecordDataset(filename, compression_type='GZIP', buffer_size=BUFFER_SIZE)
dataset = dataset.map(parser_fn)
# dataset = dataset.take(1).map(cluster_maker)
print(dataset)

In [ ]:
# initial values
TENERGY = 500
TLOCMAX = 350
TNUMBER = 3
# good values to test if recursive function works
TSEED = tf.constant(100, tf.float32)  # tf.constant([300.], tf.float32)
TNEIGHBOR = tf.constant(50., tf.float32)
TCELL = tf.constant(10., tf.float32)

In [ ]:
# EM 633
TENERGY = 5000
TLOCMAX = 500
TNUMBER = 3
# good values to test if recursive function works
TSEED = tf.constant(6., tf.float32)  # tf.constant([300.], tf.float32)
TNEIGHBOR = tf.constant(3., tf.float32)
TCELL = tf.constant(3., tf.float32)

In [ ]:
# # Had 420
# TENERGY = 0
# TLOCMAX = 500
# TNUMBER = 3
# # good values to test if recursive function works
# TSEED = tf.constant(4., tf.float32)  # tf.constant([300.], tf.float32)
# TNEIGHBOR = tf.constant(2., tf.float32)
# TCELL = tf.constant(0., tf.float32)

In [ ]:
cols = ['eventId', 'cluster_comi', 'cluster_comj', 'S_sum', 'S_rad_mean', 'S_hot', 'C_sum', 'C_rad_mean', 'C_hot']
pdf = pd.DataFrame(columns=cols)
i = 0
for parsed in dataset.take(10):
    print(i)
    output = cluster_maker(parsed)
    output2 = cluster_splitter(output)
    output3 = scalar_features(output2)
    s = parsed['scalar'].numpy()
    eventId = parsed['eventId'].numpy()
    eventId_rep = np.tile(eventId, [s.shape[0], 1])
    arr = np.hstack((eventId_rep, s))
    pdfi = pd.DataFrame(arr, columns=cols)
    pdf = pdf.append(pdfi)
    i += 1
pdf.reset_index(drop=True, inplace=True)
pdf.eventId = pdf.eventId.astype(int)
pdf.to_csv('clustering.csv', index=False)

In [ ]:
pdf

In [ ]:
pdf = pd.DataFrame(columns=['comi', 'comj', 'S_sum', 'S_rad_mean', 'S_hot', 'C_sum', 'S_rad_mean', 'C_hot'])
fig, ax = plt.subplots(ncols=2, nrows=5, figsize=(10, 30))
i = 0
for parsed_record in dataset.take(1):
    plt.imshow(parsed_record['image'][:,:,0])
    plt.colorbar()
#     print(parsed_record['labels'])
#     output = cluster_maker(parsed_record)
#     A = output['proto'].numpy()
#     A1 = A[A[...,3] == 0][...,:3]
#     A2 = A[A[...,3] == 1][...,:3]
#     # A1 = np.unique(A1, axis=0)
#     # A2 = np.unique(A2, axis=0)
#     # print(A1)
#     sp = tf.SparseTensor(A1[...,1:3], A1[...,0], dense_shape=[HEIGHT,WIDTH])
#     A1_dense = tf.sparse.to_dense(tf.sparse.reorder(sp))
#     sp = tf.SparseTensor(A2[...,1:3], A2[...,0], dense_shape=[HEIGHT,WIDTH])
#     A2_dense = tf.sparse.to_dense(tf.sparse.reorder(sp))
#     ax[i][0].imshow(A1_dense)
#     ax[i][1].imshow(A2_dense)
    
#     output2 = cluster_splitter(output)
# #     A = output['cluster'].numpy()
# #     A1 = A[A[...,3] == 0][...,:3]
# #     A2 = A[A[...,3] == 1][...,:3]
# # #     # A1 = np.unique(A1, axis=0)
# # #     # A2 = np.unique(A2, axis=0)

#     output3 = scalar_features(output2)
#     print(parsed_record['scalar'])
# #     u = np.unique(A1[:,0])
# #     print(u)
    
# #     sp = tf.SparseTensor(A1[...,1:3], A1[...,0], dense_shape=[HEIGHT,WIDTH])
# #     A1_dense = tf.sparse.to_dense(tf.sparse.reorder(sp))
# #     ax[i][1].imshow(A1_dense)
    
#     i += 1
#     print("eventId", parsed_record['eventId'])